In [2]:
import csv
import numpy as np
import pickle

LABELS = {
    'No Finding': 0,
    'Atelectasis': 1,
    'Cardiomegaly': 2,
    'Consolidation': 3,
    'Edema': 4,
    'Effusion': 5,
    'Emphysema': 6,
    'Fibrosis': 7,
    'Hernia': 8,
    'Infiltration': 9,
    'Infiltrate': 9,
    'Mass': 10,
    'Nodule': 11,
    'Pleural_Thickening': 12,
    'Pneumonia': 13,
    'Pneumothorax': 14
}

PRIORS = np.array([[113.26, 94.50], [673.04, 494.74], [216.05, 218.86],
                   [495.50, 343.66], [239.74, 490.62]])

# reduce to X by X
TARGET_GRID = 8
CLASSES = 15
ORIGINAL = 1024
BLOCKSIZE = ORIGINAL // TARGET_GRID
BOXES = 5

data = {}
with open('data/BBox_List_2017.csv') as f:
    re = csv.reader(f)
    next(re)
    for r in re:
        id = r[0]
        if id not in data:
            data[id] = np.zeros([TARGET_GRID, TARGET_GRID, BOXES, 5 + CLASSES])
        obs = LABELS[r[1]] + 5

        x = float(r[2])
        x_block = int(x // BLOCKSIZE)
        x_offset = (x % BLOCKSIZE) / BLOCKSIZE

        y = float(r[3])
        y_block = int(y // BLOCKSIZE)
        y_offset = (y % BLOCKSIZE) / BLOCKSIZE

        # exp(pred_w) * prior = w
        w = float(r[4])
        w = w / PRIORS[:, 0]
        h = float(r[5])
        h = h / PRIORS[:, 1]

        # print(x, y)
        # print(x_block, y_block)
        # print(x_offset, y_offset)

        # conf
        data[id][x_block, y_block, :, 0] = 1
        # x y
        data[id][x_block, y_block, :, 1] = x_offset
        data[id][x_block, y_block, :, 2] = y_offset
        # w h
        data[id][x_block, y_block, :, 3] = w
        data[id][x_block, y_block, :, 4] = h
        # classes
        data[id][x_block, y_block, :, obs] = 1


with open('data/labels_boxed.pkl', 'wb') as f:
    pickle.dump(data, f)

In [10]:
with open('data/labels_boxed.pkl', 'rb') as f:
    data_labels_boxed =  pickle.load(f)
print(data_labels_boxed['00013118_008.png'].shape)

(8, 8, 5, 20)


In [4]:
import csv
import pickle
 
LABELS = {
    'No Finding': 0,
    'Atelectasis': 1,
    'Cardiomegaly': 2,
    'Consolidation': -1,
    'Edema': -1,
    'Effusion': 3,
    'Emphysema': -1,
    'Fibrosis': -1,
    'Hernia': -1,
    'Infiltration': -1,
    'Infiltrate': 4,
    'Mass': 5,
    'Nodule': 6,
    'Pleural_Thickening': -1,
    'Pneumonia': 7,
    'Pneumothorax': 8
}
 
testset = set()
validset = set()
trainset = set()
 
with open('data/train.txt') as f:
    re = csv.reader(f)
    for r in re:
        trainset.add(r[0])
with open('data/valid.txt') as f:
    re = csv.reader(f)
    for r in re:
        validset.add(r[0])
with open('data/test.txt') as f:
    re = csv.reader(f)
    for r in re:
        testset.add(r[0])
traindata = {}
validdata = {}
testdata = {}
 
with open('data/Data_Entry_2017_v2.csv') as f:
    re = csv.reader(f)
    next(re)
    for r in re:
        id = r[0]
        obs = []
        for observe in r[1].split('|'):
            ob = LABELS[observe]
            if ob == -1:
                continue
            obs.append(ob)
        if len(obs) == 0:
            continue
        if id in trainset:
            traindata[id] = obs
        elif id in validset:
            validdata[id] = obs
        elif id in testset:
            testdata[id] = obs
with open('data/pickles/labels_train.pkl', 'wb') as f:
    pickle.dump(traindata, f)
with open('data/pickles/labels_valid.pkl', 'wb') as f:
    pickle.dump(validdata, f)
with open('data/pickles/labels_test.pkl', 'wb') as f:
    pickle.dump(testdata, f)

In [11]:
from keras.layers import Conv2D, BatchNormalization, Input, Activation
from keras.layers import LeakyReLU, Lambda, Reshape, Concatenate, Add, Dense

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import keras.backend as K
from keras.models import Model
import pickle
# from skimage.io import imread
# from skimage import img_as_float
# from skimage.transform import rescale
import numpy as np
import os
import sys

TRAINSIZE = 111240

EPOCHS = 20
STEPSPEREPOCH = 1024
VALIDATIONSTEPS = 2
BATCHSIZE = 64

INPUT_SIZE = 256

OUTPUT_GRID = 8
BOXES = 5
CLASSES = 15
FINAL_DIMS = BOXES * (5 + CLASSES)

MODELDIR = 'model/' + os.path.basename(os.path.splitext(sys.argv[0])[0])
MODELFILE = MODELDIR + '/model'

tb = TensorBoard(log_dir=MODELDIR)
sv = ModelCheckpoint(MODELFILE, save_best_only=True, save_weights_only=True)
es = EarlyStopping(patience=3)


def main():
    model = create_model()

    def g():
        X, Y, W = [], [], []
        while True:
            if len(X) < BATCHSIZE * 0.1:
                x, y, w = next(boxedgen())
            else:
                x, y, w = next(labelgen())
            X.append(x)
            Y.append(y)
            W.append(w)
            if len(X) == BATCHSIZE:
                Y = np.stack(Y)
                Y = [Y[..., 0:1], Y[..., 1:3], Y[..., 3:5], Y[..., 5:]]
                yield np.stack(X), Y, [
                    np.squeeze(w) for w in np.hsplit(np.array(W), 4)
                ]
                X, Y, W = [], [], []

    def vg():
        X, Y, W = [], [], []
        while True:
            x, y, w = next(boxedgen())
            # x, y = next(validgen())
            # w = [0, 0, 0, 1]
            X.append(x)
            Y.append(y)
            W.append(w)
            if len(X) == BATCHSIZE:
                Y = np.stack(Y)
                Y = [Y[..., 0:1], Y[..., 1:3], Y[..., 3:5], Y[..., 5:]]
                yield np.stack(X), Y, [
                    np.squeeze(w) for w in np.hsplit(np.array(W), 4)
                ]
                X, Y, W = [], [], []

    model.fit_generator(
        g(),
        steps_per_epoch=STEPSPEREPOCH,
        validation_data=vg(),
        validation_steps=VALIDATIONSTEPS,
        epochs=EPOCHS,
        workers=3,
        use_multiprocessing=True,
        shuffle=True,
        callbacks=[tb, sv, es])


def create_model():
    def conv(x, filters, strides=1):
        x = Conv2D(filters, 3, strides=strides, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        return x

    def space_to_depth(x):
        ss = x.shape[1] // 2
        ldim = x.shape[-1]
        x = K.reshape(x, [-1, ss, ss, ss, ss, ldim])
        x = K.permute_dimensions(x, [0, 1, 3, 2, 4, 5])
        x = K.reshape(x, [-1, ss, ss, ldim * 4])
        return x

    xi = Input([INPUT_SIZE, INPUT_SIZE])

    x = Reshape([INPUT_SIZE, INPUT_SIZE, 1])(xi)

    x = conv(x, 32)
    x = conv(x, 32, 2)

    x = conv(x, 32)
    x = conv(x, 32, 2)

    x = conv(x, 64)
    x = conv(x, 64, 2)

    x = conv(x, 64)
    x = conv(x, 64, 2)

    passthrough = Lambda(space_to_depth)(x)

    x = conv(x, 128)
    x = conv(x, 128, 2)
    x = conv(x, 128)

    x = Concatenate()([x, passthrough])

    x = conv(x, 256)
    x = conv(x, 256)

    x = Conv2D(FINAL_DIMS, 3, padding='same')(x)
    x = Reshape([OUTPUT_GRID, OUTPUT_GRID, BOXES, 5 + CLASSES])(x)

    # ============ Output
    conf = Lambda(lambda x: x[:, :, :, :, 0:1])(x)
    conf = Activation('sigmoid', name='c')(conf)

    xy = Lambda(lambda x: x[:, :, :, :, 1:3])(x)
    xy = Activation('sigmoid', name='xy')(xy)

    # wh = Lambda(lambda x: K.exp(x[:, :, :, :, 3:5]))(x)
    wh = Lambda(lambda x: x[:, :, :, :, 3:5])(x)
    wh = Activation('tanh')(wh)
    ones = Lambda(lambda x: K.ones_like(x))(wh)
    wh = Add(name='wh')([wh, ones])

    prob = Lambda(lambda x: x[:, :, :, :, 5:])(x)
    prob = Activation('sigmoid', name='p')(prob)

    # ============ Model
    model = Model(xi, [conf, xy, wh, prob])
    model.compile(
        'adam', ['binary_crossentropy', 'mse', 'mse', 'binary_crossentropy'],
        loss_weights=[1, 5, 5, 1])
    model.summary()
    return model


def boxedgen():
    with open('data/pickles/labels_boxed.pkl', 'rb') as f:
        boxed = pickle.load(f)
    while True:
        for k, v in boxed.items():
            yield img_as_float(rescale(imread('data/images/' + k),
                                       0.25)), v, [0.25, 0.25, 0.25, 0.25]


def labelgen():
    with open('data/pickles/labels_train.pkl', 'rb') as f:
        traindata = pickle.load(f)
    while True:
        for k, v in traindata.items():
            lab = np.zeros([OUTPUT_GRID, OUTPUT_GRID, BOXES, 5 + CLASSES])
            for obs in v:
                lab[:, :, :, obs] = 1
            yield img_as_float(rescale(imread('data/images/' + k),
                                       0.25)), lab, [0, 0, 0, 1]


def validgen():
    with open('data/pickles/labels_valid.pkl', 'rb') as f:
        traindata = pickle.load(f)
    while True:
        for k, v in traindata.items():
            lab = np.zeros([OUTPUT_GRID, OUTPUT_GRID, BOXES, 5 + CLASSES])
            for obs in v:
                lab[:, :, :, obs] = 1
            yield img_as_float(rescale(imread('data/images/' + k),
                                       0.25)), lab, [0, 0, 0, 1]

In [4]:
from keras.applications.resnet50 import ResNet50

def create_model_res():

    xi = Input([INPUT_SIZE, INPUT_SIZE, 3])
    x = Reshape([INPUT_SIZE, INPUT_SIZE, 3])(xi)

    ir = ResNet50(
        include_top=False,
        weights='imagenet',
        input_tensor=x,
        input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
        pooling='avg')

    p = Dense(CLASSES, activation='softmax')(ir.output)

    # ============ Model
    model = Model(ir.input, p)
    model.compile('adam', 'binary_crossentropy')
    model.summary()
    return model

In [17]:
model = create_model_res()

NameError: name 'Dense' is not defined

In [42]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
CLASSES = 15
def create_model_ResNetV2():
    INPUT_SIZE = 256
    xi = Input([INPUT_SIZE, INPUT_SIZE, 3 ])
    x = Reshape([INPUT_SIZE, INPUT_SIZE, 3])(xi)

    ir = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=x,
        input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
        pooling='avg')

    p = Dense(CLASSES, activation='softmax')(ir.output)

    # ============ Model
    model = Model(ir.input, p)
    model.compile('adam', 'binary_crossentropy')
#     model.summary()
    return model



In [ ]:
from scipy.misc import imread, imsave, imresize
from keras.utils import to_categorical
CLASSES = 15

def x_gen(batch_size, valid = False):
    if(valid):
        with open('data/pickles/labels_valid.pkl', 'rb') as f:
            traindata = pickle.load(f)
    else:
        with open('data/pickles/labels_train.pkl', 'rb') as f:
            traindata = pickle.load(f)
    X, y = [], []
    while True:
        for k, v in traindata.items():
            img = imread('data/images/' + k, mode ='RGB')
            img = img / 255
            X.append(imresize(img ,size=(256,256)))
            y.append(v[0])
            if(len(X) == batch_size):
                y = to_categorical(y, num_classes=CLASSES)
                X = np.array(X)
                yield X,y
                X, y = [], []

def train():
    model = create_model_ResNetV2()
    model.fit_generator( x_gen(32, valid = False), validation_data = x_gen(32, valid = True),steps_per_epoch=10, epochs=50, metrics=['acc'], )
    
    
train()
     


